In [0]:
# DLT works with three types of Datasets
# 1. Streaming tables(Permanent/ temporary) - Uswd as append Data Scources, Incremental data
#2.Materialized view - Used for transfermations, aggregation, aggregations or computations
#3.Views - Used for intermidiate Transformations, Not stored in Target Schema

import dlt

# Part3 of DLT

In [0]:
_order_status = spark.conf.get("custom.orderStatus","NA")

In [0]:
# Auto Loader in DLT
# Append Flow to union streaming Tables
# Pass parameters in DLT pipeline
# Generate DLT tables dynamically


In [0]:
# Create Streaming Table for Orders
@dlt.table(
    table_properties={"quality": "bronze"},
    comment = "Orders Bronze Table"
)
def orders_bronze():
    df= spark.readStream.table("dev.bronze.orders_raw")
    return df

In [0]:
# Create a Streaming Table for Autoloader
@dlt.table(
    table_properties={"quality": "bronze"},
    comment="order Autoloader Bronze Table",
    name="orders_autoloader_bronze"
)
def func_autoloader():
    df = (
    spark
    .readStream
    .format("cloudFiles")
    .option("cloudFiles.schemaHints", "o_orderkey long, o_custkey long, o_orderstatus string, o_totalprice decimal(18,2), o_orderdate date, o_orderpriority string, o_clerk string, o_shippriority integer, o_comment string")
    .option("cloudFiles.schemaLocation","/Volumes/dev/etl/landing/autoloader/schemas/1/")
    .option("cloudFiles.format", "CSV")
    .option("pathGlobfilter", "*.csv")
    .option("cloudFiles.schemaEvolutionMode", "none")
    .load('/Volumes/dev/etl/landing/files/')
    )
    return df


In [0]:
# If two streaming tables are joined together by Union, then all the data will read everytime. So avoid this and data should be read incrementally we have use appendFlow.

In [0]:
#Union of Two streaming tables order_bronze and orders_autoloader_bronze

dlt.create_streaming_table("orders_union_bronze")

#AppendFlow

@dlt.append_flow(
    target = "orders_union_bronze"
    )

def order_delta_append():
    df = spark.readStream.table("LIVE.orders_bronze")
    return df
    
@dlt.append_flow(
    target = "orders_union_bronze"
    )

def order_autoloader_append():
    df = spark.readStream.table("LIVE.orders_autoloader_bronze")
    return df
    
    

In [0]:
# Create a Materialized View for CUstomers

@dlt.table(
    table_properties={"quality": "bronze"},
    comment = "Customer Bronze Table",
    name = "customers_bronze"
)
def cust_bronze():
    df = spark.read.table("dev.bronze.customers_raw")
    return df

In [0]:
#CREATE a view to join orders with customers
@dlt.table(
    comment = "joined_view",
    
)
def joined_vw():
    df_c= spark.read.table("LIVE.customers_bronze")
    df_o= spark.read.table("LIVE.orders_union_bronze")
    df_join = df_o.join(df_c, on=df_c.c_custkey == df_o.o_custkey, how ="left_outer" )
    return df_join

In [0]:
# CREATE MV to add a new column
# Change the name of silver table from "joined_silver" to "order_silver"
from pyspark.sql.functions import current_timestamp,sum,count
@dlt.table(
    table_properties= {"quality": "silver"},
    comment = "Joined_table",
    name = "orders_silver"
)
def joined_silver():
    df= spark.read.table("LIVE.joined_vw").withColumn("_insert_date", current_timestamp())
    return df

In [0]:
#Aggregate based on c_mktsegment and find the count of order(c_orderkey)
#Added a new column and modified the a column from "sum_orders" to "count_orders"
from pyspark.sql.functions import *

for _status in _order_status.split(","):

    @dlt.table(
        table_properties={"quality": "gold"},
        comment = "Orders aggregate table",
        name = f"order_agg_{_status}_gold"
        )
    def func():
        df= spark.read.table("LIVE.orders_silver")
        df_final= df.where(f"o_orderstatus = '{_status}'").groupBy("c_mktsegment").agg(count("o_orderkey").alias ("count_orders"),sum("o_totalprice").alias("sum_totalprice")).withColumn("_insert_date", current_timestamp())
        return df_final